In [4]:
import tensorflow as tf
import logging, sys
logging.disable(sys.maxsize)

print(tf.__version__)

DIMX = 4096
DIMY = 4096

class Model(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec(shape=[DIMX, DIMY], dtype=tf.float32, name=None), 
                                  tf.TensorSpec(shape=[DIMX, DIMY], dtype=tf.float32, name=None)])
    def func1(self, x, y):
        return tf.matmul(x,y)

model = Model()

converter = tf.lite.TFLiteConverter.from_concrete_functions([model.func1.get_concrete_function()], model)
tflite_model = converter.convert() 

with open('genGEMM_float32.tflite', 'wb') as f:
    f.write(tflite_model)

interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

2.13.0
INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmpcmmer306\assets


INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmpcmmer306\assets


{'serving_default': {'inputs': ['x', 'y'], 'outputs': ['output_0']}}


In [3]:
#Integer Float Fallback Quantization
converter = tf.lite.TFLiteConverter.from_concrete_functions([model.func1.get_concrete_function()], model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset():
    for i in range(10): # create fake data
        fake_data_x = tf.random.normal([DIMX, DIMY])
        fake_data_y = tf.random.normal([DIMX, DIMY])
        yield [fake_data_y, fake_data_x]
converter.representative_dataset = representative_dataset

tflite_model_integer_fallback = converter.convert()

with open('genGEMM_integer_fallback.tflite', 'wb') as f:
  f.write(tflite_model_integer_fallback)

INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmp4di8xjsw\assets


INFO:tensorflow:Assets written to: C:\Users\Xiaojun\AppData\Local\Temp\tmp4di8xjsw\assets
